In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [32]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten/'
filename = 'privatkunden.iid.ELPHI.ohneAppleEtc_Validierung.csv'
df_relevante_iids = pd.read_csv(os.path.join(path, filename))

In [33]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten/'
filename = 'Thalia_transactions_zweiteBereinigung_Validierung.csv'
df_transactions = pd.read_csv(os.path.join(path, filename))

In [34]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Thalia_history_ersteBereinigung.csv'
df_history = pd.read_csv(os.path.join(path, filename))

#### 0. Definition, welche Monat der Startmonat ist
##### Im Hauptmodell ist der Startmonat = 17

In [35]:
startmonat = 18

#### 1. Weitere Verarbeitung Transaktionen

In [36]:
df_t_merged = pd.merge(df_transactions, df_relevante_iids, on =["iid"])

In [37]:
df_t_merged_grouped = df_t_merged.copy()
df_t_merged_grouped["month_count"] = df_t_merged_grouped["month_count"] - startmonat + 1
df_t_merged_grouped = df_t_merged_grouped.groupby(["iid", "month_count"]).agg(count=("iid", "count"))
df_t_merged_grouped = df_t_merged_grouped.reset_index()

In [38]:
df_t_merged_grouped.head()

,iid,month_count,count
0,100011954,1,2
1,100011954,3,2
2,100011954,7,1
3,100011954,11,1
4,100022607,1,1


In [39]:
df_pivoted_t = df_t_merged_grouped.pivot(index="iid", columns="month_count", values="count")
df_pivoted_t = df_pivoted_t.reset_index()
df_pivoted_t = df_pivoted_t.fillna(0)

In [40]:
df_pivoted_t.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100011954,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,100022607,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100057681,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,3.0,0.0,1.0,0.0
3,100057957,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,100064191,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


##### 1.1 LY berechnen

In [41]:
df_LY = df_pivoted_t.copy()
df_LY = df_LY.rename(columns={col: str(col) for col in df_LY.columns})
df_LY[df_LY.columns.difference(['iid'])] = df_LY[df_LY.columns.difference(['iid'])].applymap(lambda x: 1.0 if x != 0.0 else x)

In [42]:
df_LY.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100011954,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,100022607,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100057681,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
3,100057957,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,100064191,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [43]:
# neue Spaltennamen für df_LY_anwendung definieren
new_columns = ["iid"]

# leeres DataFrame mit den neuen Spaltennamen erstellen
df_LY_anwendung = pd.DataFrame(columns=new_columns)

# Spalte "iid" aus df_LY kopieren
df_LY["0"] = 1.0
df_LY_anwendung["0"] = 1.0
df_LY_anwendung["iid"] = df_LY["iid"]

# Schleife über alle Spalten außer "iid" in df_LY
# df_LY.shape[1] - 1, da doe Spalte iid niht berücksichtigt wird
for i in range(1, df_LY.shape[1] - 1):
    # die vorherige Spalte aus df_LY für den aktuellen Monat
    previous_month = i-1
    previous_month = str(previous_month)
    
    # neue Spalte für den aktuellen Monat in df_LY_anwendung
    current_month = i
    
    # Schleife über alle Zeilen in df_LY
    for index, row in df_LY.iterrows():
        # Prüfen, ob die vorherige Spalte in df_LY ungleich 0 ist
        if row[previous_month] != 0.0:
            df_LY_anwendung.at[index, str(current_month)] = 1.0
        else:
            # Wenn die vorherige Spalte in df_LY gleich 0 ist, um 1 erhöhen
            df_LY_anwendung.at[index, str(current_month)] = df_LY_anwendung.at[index, str(current_month-1)] + 1
            
df_LY_anwendung = df_LY_anwendung.drop("0", axis=1)

In [44]:
df_LY_anwendung.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100011954,1.0,1.0,2.0,1.0,2.0,3.0,4.0,1.0,2.0,3.0,4.0,1.0,2.0,3.0
1,100022607,1.0,1.0,1.0,2.0,3.0,4.0,1.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
2,100057681,1.0,1.0,1.0,1.0,2.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0
3,100057957,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,4.0,1.0,2.0,1.0
4,100064191,1.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,1.0,2.0,3.0,4.0,5.0


#### 2. Weitere Verarbeitung History

In [45]:
df_h_merged = pd.merge(df_history, df_relevante_iids, on =["iid"])

In [46]:
df_h_merged_grouped = df_h_merged.copy()
df_h_merged_grouped = df_h_merged_grouped[df_h_merged_grouped["month_count"] >= startmonat]
df_h_merged_grouped["month_count"] = df_h_merged_grouped["month_count"] - startmonat + 1

In [47]:
df_h_merged_grouped = df_h_merged_grouped.groupby(["iid", "month_count", "attrtype"]).agg(count=("iid", "count"))
df_h_merged_grouped = df_h_merged_grouped.reset_index()

##### 2.1 Pivot table für HY

In [48]:
df_h_merged_grouped_HY = df_h_merged_grouped[df_h_merged_grouped["attrtype"] == "HY"]
df_h_merged_grouped_HY = df_h_merged_grouped_HY[["iid", "month_count", "count"]]

In [49]:
df_pivoted_HY = df_h_merged_grouped_HY.pivot(index="iid", columns="month_count", values="count")
df_pivoted_HY = df_pivoted_HY.reset_index()
df_pivoted_HY = df_pivoted_HY.fillna(0)

In [50]:
df_pivoted_HY.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100057681,1.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
1,100057957,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
2,100064191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,1.0
3,100064787,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
4,100066593,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0


##### 2.2 Pivot table für NO

In [51]:
df_h_merged_grouped_NO = df_h_merged_grouped[df_h_merged_grouped["attrtype"] == "NO"]
df_h_merged_grouped_NO = df_h_merged_grouped_NO[["iid", "month_count", "count"]]

In [52]:
df_pivoted_NO = df_h_merged_grouped_NO.pivot(index="iid", columns="month_count", values="count")
df_pivoted_NO = df_pivoted_NO.reset_index()
df_pivoted_NO = df_pivoted_NO.fillna(0)

In [53]:
df_pivoted_NO.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100057681,1.0,2.0,3.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,1.0,2.0,1.0
1,100057957,2.0,2.0,3.0,2.0,3.0,2.0,0.0,2.0,1.0,1.0,2.0,2.0,3.0,0.0
2,100064191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0
3,100064787,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0
4,100066593,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0


##### 2.3 Pivot table für LO

In [54]:
df_LO = df_pivoted_NO
df_LO = df_LO.rename(columns={col: str(col) for col in df_LO.columns})
df_LO[df_LO.columns.difference(['iid'])] = df_LO[df_LO.columns.difference(['iid'])].applymap(lambda x: 1.0 if x != 0.0 else x)

In [55]:
df_LO.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100057681,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
1,100057957,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,100064191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,100064787,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,100066593,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [56]:
# neue Spaltennamen für df_LY_anwendung definieren
new_columns = ["iid"]

# leeres DataFrame mit den neuen Spaltennamen erstellen
df_LO_anwendung = pd.DataFrame(columns=new_columns)

# Spalte "iid" aus df_LY kopieren
df_LO["0"] = 1.0
df_LO_anwendung["0"] = 0.0
df_LO_anwendung["iid"] = df_LO["iid"]

# Schleife über alle Spalten außer "iid" in df_LY
for i in range(1, df_LO.shape[1]-1):
    # die vorherige Spalte aus df_LY für den aktuellen Monat
    previous_month = i-1
    
    # neue Spalte für den aktuellen Monat in df_LY_anwendung
    current_month = i
    
    # Schleife über alle Zeilen in df_LY
    for index, row in df_LO.iterrows():
        # Prüfen, ob die vorherige Spalte in df_LY ungleich 0 ist
        if row[str(previous_month)] != 0.0:
            df_LO_anwendung.at[index, str(current_month)] = 1.0
        else:
            # Wenn die vorherige Spalte in df_LY gleich 0 ist, um 1 erhöhen
            df_LO_anwendung.at[index, str(current_month)] = df_LO_anwendung.at[index, str(previous_month)] + 1
            
#Nan durch Nullen ersetzen
df_LO_anwendung = df_LO_anwendung.fillna(0)
df_LO_anwendung = df_LO_anwendung.drop("0", axis=1)

In [57]:
df_LO_anwendung.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100057681,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0
1,100057957,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
2,100064191,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,1.0,1.0,2.0,1.0
3,100064787,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,100066593,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Ausgabe der Daten

In [58]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde_Validierung.csv'

df_pivoted_t.to_csv(os.path.join(path, filename), index=False)

In [59]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde_Validierung.csv'

df_LY_anwendung.to_csv(os.path.join(path, filename), index=False)

In [60]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde_Validierung.csv'

df_pivoted_HY.to_csv(os.path.join(path, filename), index=False)

In [61]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_NO_Privat_ohneApple_Langzeitkunde_Validierung.csv'

df_pivoted_NO.to_csv(os.path.join(path, filename), index=False)

In [62]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde_Validierung.csv'

df_LO_anwendung.to_csv(os.path.join(path, filename), index=False)